In [1]:
from multiprocessing import Pool
from scipy.spatial.distance import cosine as cos_distance
from utils import mapper, valid_smiles
from utils import disable_rdkit_log, enable_rdkit_log
from utils import read_smiles_csv
from metrics.utils import compute_fragments, calc_agg_tanimoto, calc_self_tanimoto, \
    compute_scaffolds, fingerprints, canonic_smiles, get_mol, compute_scaffold
import argparse
import numpy as np
import pandas as pd
import calc_metrics
import warnings
warnings.filterwarnings("ignore")


In [1]:
import pandas as pd
import numpy as np

models = ['aae', 'vae', 'latentgan', 'reinvent', 'pocket2mol', 'moldqn', 'sbmolgen', 'lsmolgen']

for m in [7]:
    model = models[m]
    print(model)
    df = pd.read_csv('result/egfr_'+model+'_gen_0_eval_result.csv', header=None, index_col=0)
    df = pd.DataFrame(df.values.T, columns=df.index, index=df.columns)
    for i in range(5):
        df_ = pd.read_csv('result/egfr_'+model+'_gen_'+str(i+1)+'_eval_result.csv', header=None, index_col=0)
        df_ = pd.DataFrame(df_.values.T, columns=df_.index, index=df_.columns)
        df = pd.concat([df, df_])
    mean = df.mean()
    std = df.std()
    df.loc[2] = mean
    df.loc[3] = std
#     print(df)
df = df.drop(columns='Recovery/Scaf')
df

lsmolgen


,Validity,Uniqueness,Diversity,Novelty,Recovery/Frag,Active_rate,Success_rate
1,1.0,0.998200,0.980800,0.973800,0.202989,1.000000,0.957200
1,1.0,0.999400,0.989000,0.982600,0.252121,0.999600,0.961000
1,1.0,0.999000,0.970400,0.940000,0.156463,0.994400,0.937400
1,1.0,0.999000,0.975400,0.959800,0.187553,0.998400,0.941800
1,1.0,0.986600,0.976000,0.966400,0.175000,1.000000,0.949000
1,1.0,0.999200,0.983400,0.971400,0.235981,0.998800,0.955200
2,1.0,0.996900,0.979167,0.965667,0.201684,0.998533,0.950267
3,0.0,0.005062,0.006608,0.014693,0.036548,0.002127,0.009235


In [2]:
print('${:.3f}\pm {:.3f}$ &${:.3f}\pm {:.3f}$ &${:.3f}\pm {:.3f}$ &${:.3f}\pm {:.3f}$ &${:.3f}\pm {:.3f}$ &${:.3f}\pm {:.3f}$ &${:.3f}\pm {:.3f}$'.format(df['Validity'][2], df['Validity'][3], df['Uniqueness'][2], df['Uniqueness'][3], df['Diversity'][2], df['Diversity'][3], df['Novelty'][2], df['Novelty'][3], df['Recovery/Frag'][2], df['Recovery/Frag'][3], df['Active_rate'][2], df['Active_rate'][3], df['Success_rate'][2], df['Success_rate'][3]))

$1.000\pm 0.000$ &$0.997\pm 0.005$ &$0.979\pm 0.007$ &$0.966\pm 0.015$ &$0.202\pm 0.037$ &$0.999\pm 0.002$ &$0.950\pm 0.009$


In [9]:
df = pd.read_csv('data/Mpro/Mpro_chembl_article.smi')
df['Scaffold'] = df['Smiles'].apply(compute_scaffold)


In [15]:
scaf = df.Scaffold.value_counts()
scaf.keys()

Index(['O=C(CSc1c[nH]c2ccccc12)NCc1ccccc1',
       'O=c1[nH]cc(-c2cc(-c3cccc(OCc4ccccc4)c3)c(=O)n(-c3cccnc3)c2)c(=O)[nH]1',
       'O=c1[nH]cc(-c2cc(-c3ccccc3)c(=O)n(-c3cccnc3)c2)c(=O)[nH]1',
       'O=c1c(-c2ccccc2)cc(-c2ccccc2)cn1-c1cccnc1',
       'O=C(CSc1c[nH]c2ccccc12)N(Cc1ccccc1)Cc1ccccc1',
       'O=C(CNC(=O)CNC(=O)OCc1ccccc1)NCC[C@@H]1CCNC1=O',
       'O=c1[nH]cc(-c2cc(-c3cccc(OCC4CC4)c3)c(=O)n(-c3cccnc3)c2)c(=O)[nH]1',
       'O=c1[nH]cc(-c2cc(-c3cccc(OCCc4ccccc4)c3)c(=O)n(-c3cccnc3)c2)c(=O)[nH]1',
       'O=c1c(-c2ccccc2)cc(-c2ccccc2)cn1-c1cncnc1',
       'O=c1c(-c2cccc(OCCc3ccccc3)c2)cc(-c2ccccc2)cn1-c1cccnc1',
       'O=c1c(-c2cccc(OCc3ccccc3)c2)cc(-c2ccccc2)cn1-c1cccnc1',
       'O=c1c(-c2ccccc2)cc(-c2ccccn2)cn1-c1ccccc1',
       'O=C(CNC(=O)OCc1ccccc1)NCC[C@@H]1CCNC1=O',
       'O=C(CNC(=O)c1cc2ccccc2[nH]1)NCC[C@@H]1CCNC1=O',
       'O=[PH](OC[C@@H]1CCC(c2ccc3cncnn23)O1)Oc1ccccc1',
       'O=C(CSc1c[nH]c2c(-c3ccccc3)cccc12)NCc1ccccc1',
       'O=C(CC1SC(NC(c2ccccc2)c2c[n

In [28]:
from rdkit.Chem import Draw

for i, s in enumerate(scaf.keys()[:5]):
    img = Draw.MolToImage(Chem.MolFromSmiles(s), size=(300, 300))
    img.save("./figure/Mpro_bioactive_ligands_scaffold"+str(i)+".png")

# Draw.MolsToGridImage([Chem.MolFromSmiles(x) for x in scaf.keys()[:6]], molsPerRow=3, subImgSize=(400,480))
# scaf.values

In [10]:
import scaffoldgraph as sg
import networkx as nx
import matplotlib.pyplot as plt
from rdkit.Chem import Draw
from rdkit import Chem
import random
import os
from collections import Counter

# 统计分子骨架数量并展示Top的分子
mols = list(df_mv411['mol'].values)
mol_scaffolds = []
for mol in mols:
    frags = sg.tree_frags_from_mol(mol)
    if len(frags)>1:
        frag = frags[-2]
    else:
        frag = frags[-1]
    if frag.GetNumAtoms():
        mol_scaffolds.append(frag)

smi = [Chem.MolToSmiles(mol) for mol in mol_scaffolds]
counter = Counter(smi)
sort_counter = sorted(counter.items(),  key=lambda d: d[1], reverse=True)

key = [sort_counter[i][0] for i in range(len(sort_counter))]
value = [sort_counter[i][1] for i in range(len(sort_counter))]

print(len(key))
Draw.MolsToGridImage([Chem.MolFromSmiles(x) for x in key[:12]], molsPerRow=4)

,Smiles,Scaffold
0,O=C(CSc1c[nH]c2cc(Br)ccc12)NCc1ccc(Cl)c(Cl)c1,O=C(CSc1c[nH]c2ccccc12)NCc1ccccc1
1,CCOc1ccccc1CN(OC)C(=O)CSc1c[nH]c2cccc(Br)c12,O=C(CSc1c[nH]c2ccccc12)NCc1ccccc1
2,Cn1cc(SCC(=O)NCc2ccc(Cl)c(Cl)c2)c2ccccc21,O=C(CSc1c[nH]c2ccccc12)NCc1ccccc1
3,CC(C)C[C@H](NC(=O)OCc1ccccc1)C(=O)N[C@@H](C[C@...,O=C(CNC(=O)OCc1ccccc1)NCC[C@@H]1CCNC1=O
4,CCOC(=O)/C=C/[C@H](C[C@@H]1CCNC1=O)NC(=O)[C@H]...,O=C(CNC(=O)CNC(=O)OCc1ccccc1)NCC[C@@H]1CCNC1=O
5,CC(C)(C)C(=O)N[C@H](C(=O)N1C[C@H]2[C@@H]([C@H]...,O=C(NCCC1CCC1)[C@H]1NC[C@@H]2C[C@H]12
6,O=C(CSc1c[nH]c2ccc(Br)cc12)NCc1ccc(Cl)c(Cl)c1,O=C(CSc1c[nH]c2ccccc12)NCc1ccccc1
7,C[C@@H](NC(=O)CSc1c[nH]c2ccccc12)c1ccccc1,O=C(CSc1c[nH]c2ccccc12)NCc1ccccc1
8,CC(C)C[C@H](NC(=O)OCc1ccccc1)C(=O)N[C@@H](CC(C...,O=C(CNC(=O)CNC(=O)OCc1ccccc1)NCC[C@@H]1CCNC1=O
9,O=C(CSc1c[nH]c2cccc(Br)c12)NCc1ccc(Cl)c(Cl)c1,O=C(CSc1c[nH]c2ccccc12)NCc1ccccc1


In [6]:
calc_metrics.internal_diversity(gen)

0.8244703333097839

In [1]:
import numpy as np
from metrics.utils import compute_fragments, calc_self_tanimoto

# Sample input vectors
gen_vecs = np.random.rand(500, 10)
stock_vecs = np.random.rand(1000, 10)

# Compute Tanimoto similarity using max aggregation
agg_tanimoto_max = calc_self_tanimoto(gen_vecs, agg='max')


# Print results
print(agg_tanimoto_max)

[0.416  0.5146 0.4114 0.545  0.349  0.5103 0.4084 0.459  0.4602 0.6226
 0.5566 0.4844 0.4146 0.522  0.4626 0.5083 0.572  0.545  0.5146 0.4783
 0.601  0.5063 0.4856 0.441  0.609  0.5454 0.514  0.5103 0.548  0.612
 0.508  0.3953 0.4292 0.4243 0.4548 0.4453 0.513  0.4639 0.4565 0.503
 0.5615 0.5293 0.4783 0.3818 0.5244 0.484  0.5015 0.4927 0.3877 0.5044
 0.5986 0.4004 0.449  0.4663 0.5107 0.4744 0.422  0.4407 0.5605 0.4602
 0.5835 0.5757 0.4521 0.5835 0.59   0.5093 0.3745 0.535  0.3257 0.5063
 0.5054 0.5674 0.3818 0.597  0.4697 0.522  0.481  0.546  0.4932 0.53
 0.577  0.5015 0.554  0.511  0.5117 0.4666 0.5093 0.5254 0.4387 0.559
 0.6045 0.5186 0.4705 0.6113 0.482  0.488  0.4153 0.414  0.4277 0.4265
 0.5303 0.6353 0.4653 0.4578 0.4175 0.4875 0.5166 0.519  0.4924 0.555
 0.493  0.4736 0.4832 0.5317 0.4797 0.4844 0.6113 0.486  0.397  0.4988
 0.567  0.524  0.504  0.4653 0.3914 0.4102 0.5664 0.4966 0.4165 0.4067
 0.539  0.577  0.523  0.4983 0.565  0.5146 0.4514 0.5327 0.5024 0.5034
 0.4478 0.50

In [10]:
import MDAnalysis as mda
import numpy as np

def read_pdb_file(file_path):
    u = mda.Universe(file_path)
    coordinates = u.atoms.positions
    edges = []
    for atom1 in u.atoms:
        print('Atom1: ', atom1)
        for atom2 in atom1.bonded_atoms:
            print('Atom2: ', atom2)
#             distance = np.linalg.norm(atom1.position - atom2.position)
#             if distance < 4.5 or atom1.is_bonded(atom2):
#                 edges.append((atom1.index, atom2.index))
    return 
#     return {'coordinates': coordinates, 'edges': edges}


In [11]:
dic = read_pdb_file('data/EGFR/EGFR_2RGP.pdb')

Atom1:  <Atom 1: N of type N of resname ALA, resid 702 and segid A and altLoc >
Atom1:  <Atom 2: CA of type C of resname ALA, resid 702 and segid A and altLoc >
Atom1:  <Atom 3: C of type C of resname ALA, resid 702 and segid A and altLoc >
Atom1:  <Atom 4: O of type O of resname ALA, resid 702 and segid A and altLoc >
Atom1:  <Atom 5: CB of type C of resname ALA, resid 702 and segid A and altLoc >
Atom1:  <Atom 6: N of type N of resname LEU, resid 703 and segid A and altLoc >
Atom1:  <Atom 7: CA of type C of resname LEU, resid 703 and segid A and altLoc >
Atom1:  <Atom 8: C of type C of resname LEU, resid 703 and segid A and altLoc >
Atom1:  <Atom 9: O of type O of resname LEU, resid 703 and segid A and altLoc >
Atom1:  <Atom 10: CB of type C of resname LEU, resid 703 and segid A and altLoc >
Atom1:  <Atom 11: CG of type C of resname LEU, resid 703 and segid A and altLoc >
Atom1:  <Atom 12: CD1 of type C of resname LEU, resid 703 and segid A and altLoc >
Atom1:  <Atom 13: CD2 of type 

In [7]:
dic


{'coordinates': array([[  1.538,  37.455, 107.874],
        [  2.992,  37.638, 108.152],
        [  3.703,  38.126, 106.896],
        ...,
        [ 43.341,  38.833,  88.804],
        [  7.264,  51.526, 104.061],
        [  0.919,  34.57 , 108.316]], dtype=float32),
 'edges': [(2289, 2290),
  (2289, 2291),
  (2289, 2292),
  (2289, 2293),
  (2290, 2289),
  (2291, 2289),
  (2292, 2289),
  (2293, 2289),
  (2294, 2295),
  (2294, 2296),
  (2294, 2297),
  (2294, 2298),
  (2295, 2294),
  (2296, 2294),
  (2297, 2294),
  (2298, 2294),
  (2299, 2300),
  (2299, 2320),
  (2300, 2299),
  (2300, 2301),
  (2301, 2300),
  (2301, 2318),
  (2302, 2311),
  (2303, 2318),
  (2303, 2322),
  (2304, 2305),
  (2304, 2323),
  (2305, 2304),
  (2305, 2306),
  (2305, 2310),
  (2306, 2305),
  (2306, 2307),
  (2306, 2322),
  (2307, 2306),
  (2307, 2308),
  (2308, 2307),
  (2308, 2309),
  (2309, 2308),
  (2309, 2310),
  (2309, 2324),
  (2310, 2305),
  (2310, 2309),
  (2311, 2302),
  (2311, 2312),
  (2311, 2325),
  (2